In [1]:
import pandas as pd 
import re 
import numpy as np 
import tabula
import os

In [2]:
pdfs = sorted([i for i in os.listdir() if i.endswith('.pdf')])

In [3]:
dfs = tabula.read_pdf(pdfs[0], pages = 1)

In [4]:
df_parsingIncomeBracket = dfs[0]

In [6]:
indexIncomeRow = list(df_parsingIncomeBracket.index[df_parsingIncomeBracket['Unnamed: 0'].str.get(-1) == ':'])

In [8]:
income = list(df_parsingIncomeBracket.iloc[indexIncomeRow,0])

In [10]:
fruitNames = ['total fruit', 'citrus, melons, berries', 'other fruit', 'fruit juice']

In [127]:
fruits = list()
yrs = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 1)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    indexIncomeRow = a.index[a['Unnamed: 0']==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row 
    fruits.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [128]:
for dfIndex, df in enumerate(fruits): 
# loop through each data frame in fruits list 
    df.reset_index(drop = True, inplace = True)
    # reset the index since we dropped multiple rows on top 
    partition = list(df.index[df['Unnamed: 0'].isin(income)])
    # the rows whose cells are the genders 
    df['income'] = np.nan
    # gender column is created
    for i in range(len(partition)-1):
    # for each gender
        df.iloc[partition[i]: partition[i+1],1] = income[i]
        # fill in the respective gender to the resepctive rows 
        if i ==2: 
            df.iloc[partition[i]: df.shape[0],1] = income[i]
            
    df.income[df.income.isna()] = income[2]
    # the rest of the gender column is filled with 'Male and female'
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[partition], axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(fruitNames)):
    # loop through each fruit names 
        df[fruitNames[index]] = df['values'].apply(lambda x: x.split()[index*2+1])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[[0,3]], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [129]:
def parsing15_18(df, yr, element): 
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    df.dropna(inplace = True)
    df.drop([df.columns[1]], axis = 1, inplace = True)
    df.columns = ['age']+element+['income']
    df.age = df.age.apply(lambda x: x.rstrip('... '))
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    for col in element: 
        df[col] = df[col].apply(lambda x: x.split(' (')[0])
    df['year'] = [yr]*df.shape[0]
    return df

In [130]:
for i in pdfs[-2:]: 
    dfs = tabula.read_pdf(i, pages = 1)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    indexIncomeRow = a.index[a['Unnamed: 0']==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row 
    yr = re.findall('\d+_\d+', i)[0]
    
    df = parsing15_18(a, yr, fruitNames)
    
    fruits.append(df)
    # append the parsed data frame into fruits list 

In [131]:
fruits_df = pd.concat(fruits)

In [132]:
fruits_df 

,income,age,total fruit,"citrus, melons, berries",other fruit,fruit juice,year
1,"$0 - $24,999",2 -5,1.55,0.13,0.59,0.83,2005_06
2,"$0 - $24,999",6 - 11,1.22,0.14,0.48,0.60,2005_06
3,"$0 - $24,999",12 - 19,0.97,0.16,0.32,0.49,2005_06
4,"$0 - $24,999",20 and over,0.97,0.17,0.41,0.39,2005_06
5,"$0 - $24,999",2 and over,1.03,0.16,0.42,0.45,2005_06
...,...,...,...,...,...,...,...
23,All Individuals,2 -5,1.53,0.32,0.64,0.56,2017_18
24,All Individuals,6 - 11,1.17,0.32,0.55,0.30,2017_18
25,All Individuals,12 - 19,0.84,0.16,0.42,0.26,2017_18
26,All Individuals,20 and over,0.88,0.24,0.44,0.20,2017_18


## Parsing Vegetable Pages

In [38]:
veg =['total vegetables', 'total starchy', 'potatoes', 'other starchy', 'total red and orange',\
     'tomatoes', 'other red and orange','dark green','other']

In [35]:
vegetable = list()
yrs = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 2)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    indexIncomeRow = a.index[a['Unnamed: 0']==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row
    a['values'] = a.iloc[:,0]+' '+a.iloc[:,1]
    # concatenate two columns to ease the process of splitting values 
    a.drop(a.columns[[0,1]], axis = 1, inplace = True)
    # drop the columns after concatenation
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    vegetable.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [39]:
for dfIndex, df in enumerate(vegetable): 
# loop through each data frame in fruits list 
    partition = list(df.index[df['values'].isna()])
    # the rows whose cells are the genders 
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    # gender column is created
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.dropna(axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['values'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['values'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(veg)):
    # loop through each fruit names 
        df[veg[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

## Parsing Grain Pages

In [54]:
grainNames = ['total grains', 'whole grains', 'refined grains']

In [72]:
grains = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 3)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    indexIncomeRow = a.index[a['Unnamed: 0']==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row 
    grains.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [73]:
for dfIndex, df in enumerate(grains): 
# loop through each data frame in fruits list 
    df.reset_index(drop = True, inplace = True)
    incomeRows = list(df.index[df['Unnamed: 0'].str.get(-1)==':'])
    # the rows whose cells are the income
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    # gender column is created
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[incomeRows], inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(grainNames)):
    # loop through each fruit names 
        df[grainNames[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(['values', df.columns[0]], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

## Parsing Dairy Pages

In [87]:
dairyNames = ['total dairy', 'fluid milk', 'cheese', 'yoghurt']

In [88]:
daries = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 4)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    indexIncomeRow = a.index[a['Unnamed: 0']==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row 
    daries.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [89]:
for dfIndex, df in enumerate(daries): 
# loop through each data frame in fruits list 
    df.reset_index(drop = True, inplace = True)
    incomeRows = list(df.index[df['Unnamed: 0'].str.get(-1)==':'])
    # the rows whose cells are the income
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    # gender column is created
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[incomeRows], inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['Unnamed: 0'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(dairyNames)):
    # loop through each fruit names 
        df[dairyNames[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(['values', df.columns[0]], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [137]:
df

,age,total fruit,"citrus, melons, berries",other fruit,fruit juice,income,year
5,2 -5,1.40,0.17,0.53,0.70,"$0 - $24,999",2017_18
6,6 - 11,0.95,0.13,0.53,0.29,"$0 - $24,999",2017_18
7,12 - 19,0.81,0.14,0.39,0.27,"$0 - $24,999",2017_18
8,20 and over,0.80,0.17,0.39,0.24,"$0 - $24,999",2017_18
9,2 and over,0.84,0.16,0.41,0.27,"$0 - $24,999",2017_18
11,2 -5,1.43,0.43,0.57,0.43,"$25,000 - $74,999",2017_18
12,6 - 11,1.06,0.31,0.46,0.30,"$25,000 - $74,999",2017_18
13,12 - 19,0.84,0.18*,0.43,0.23,"$25,000 - $74,999",2017_18
14,20 and over,0.92,0.26,0.47,0.19,"$25,000 - $74,999",2017_18
15,2 and over,0.95,0.27,0.47,0.21,"$25,000 - $74,999",2017_18


## Parse Oil and Others

In [93]:
oil = ['oil', 'solid fats', 'added sugars', 'alcoholic drinks']

In [102]:
oilDfs = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 8)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    indexIncomeRow = a.index[a[a.columns[0]]==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row 
    oilDfs.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [103]:
for dfIndex, df in enumerate(oilDfs): 
# loop through each data frame in fruits list 
    df.reset_index(drop = True, inplace = True)
    incomeRows = list(df.index[df[df.columns[0]].str.get(-1)==':'])
    # the rows whose cells are the income
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    # gender column is created
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[incomeRows], inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df[df.columns[0]].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df[df.columns[0]].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(oil)):
    # loop through each fruit names 
        df[oil[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(['values', df.columns[0]], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

## Parsing Protein Food

In [112]:
p2 = ['eggs', 'nuts and seeds', 'soybean products']

In [113]:
protein2DFs = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 6)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    a.dropna(axis = 1, inplace = True)
    # drop the NA column 
    indexIncomeRow = a.index[a[a.columns[0]]==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row 
    protein2DFs .append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [114]:
for dfIndex, df in enumerate(protein2DFs): 
# loop through each data frame in fruits list 
    df.reset_index(drop = True, inplace = True)
    incomeRows = list(df.index[df[df.columns[0]].str.get(-1)==':'])
    # the rows whose cells are the income
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    # gender column is created
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.drop(df.index[incomeRows], inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df[df.columns[0]].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df[df.columns[0]].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(p2)):
    # loop through each fruit names 
        df[p2[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(['values', df.columns[0]], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [116]:
protein1 = ['total protein foods', 'Total Meat, Poultry, and Seafood', 'meat', 'poultry', 'cured meat',\
           'seafood low n-3', 'seafood high n-3', 'organ meat']

In [117]:
protein1DFs = list()
yrs = list()

for i in range(len(pdfs)-2): 
    dfs = tabula.read_pdf(pdfs[i], pages = 2)
    # parsing the first page of each PDF, which is the fruit intake of each year
    a = dfs[0]

    indexIncomeRow = a.index[a['Unnamed: 0']==income[0]][0]
    # the rows with income bracket, whose strings are ending with ':'
    a.drop(a.index[range(0,indexIncomeRow)],axis = 0, inplace = True)
    # drop all the rows before the first income row
    a['values'] = a.iloc[:,0]+' '+a.iloc[:,1]
    # concatenate two columns to ease the process of splitting values 
    a.drop(a.columns[[0,1]], axis = 1, inplace = True)
    # drop the columns after concatenation
    a.reset_index(drop = True, inplace = True)
    # reset the indexes after dropping rows 
    protein1DFs.append(a)
    # append the parsed data frame into fruits list 
    yrs.append(re.findall('\d+_\d+', pdfs[i]))
    # save the year in the order of the data frame 
    # this is for the year column for each data frame later on 

In [119]:
for dfIndex, df in enumerate(protein1DFs): 
# loop through each data frame in fruits list 
    partition = list(df.index[df['values'].isna()])
    # the rows whose cells are the genders 
    df['income'] = [income[0]]*6+[income[1]]*6+[income[2]]*6+[income[3]]*6
    # gender column is created
    
    df.income = df.income.apply(lambda x: x.rstrip(':'))
    # strip out ':' at the end of all the genders 
    df.dropna(axis = 0, inplace = True)
    # drop the gender rows that are not needed anymore 
    
    df['age'] = df['values'].apply(lambda x: x.split('... ')[0])
    # split the first column with '...'
    # the elements on the left of the '...' are the age groups
    df.age = df.age.apply(lambda x: x.rstrip('...'))
    # strip out '...' from each age group values
    df['values'] = df['values'].apply(lambda x: x.split('... ')[1])
    # all the numeric values are on the right side of '... '
    # create a seperate column for these values 
    
    for index in range(len(protein1)):
    # loop through each fruit names 
        df[protein1[index]] = df['values'].apply(lambda x: x.split()[index*2])
        # creating column for each fruit category 
        # splitting the rows with white space 
        # each odd-numbered element is the values we need 
        # assign the intake value of each fruit accordingly 
        
    df.drop(df.columns[0], axis = 1, inplace = True)
    # drop the columns we do not need anymore 
    df['year'] = yrs[dfIndex]*df.shape[0]
    # fill in the year column 

In [125]:
protein1DF = pd.concat(protein1DFs)
proteinDF = pd.merge(protein2DFs, protein1DF, on=["year", "age",'income'])

In [141]:
oilDF = pd.concat(oilDfs)
dairyDF = pd.concat(daries)
grainDF = pd.concat(grains)
vegDF = pd.concat(vegetable)
fruitDF = pd.concat(fruits)

In [142]:
tb_1 = pd.merge(oilDF, proteinDF, on=["year", "age",'income'])
tb_2 = pd.merge(dairyDF, grainDF, on=["year", "age",'income'])
tb_3 = pd.merge(vegDF, fruitDF, on=["year", "age",'income'])
tb_1_2 = pd.merge(tb_1, tb_2, on=["year", "age",'income'])
finalTB = pd.merge(tb_1_2, tb_3, on=["year", "age",'income'])

In [144]:
finalTB.to_csv('us_FoodG_income.csv', index = False)